## Preprocessing

In [38]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [39]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(["EIN","NAME"],1)


<ipython-input-39-3edd94d2eb9e>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df=application_df.drop(["EIN","NAME"],1)


In [40]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [41]:
# Look at APPLICATION_TYPE value counts for binning
a_counts=application_df["APPLICATION_TYPE"].value_counts()
a_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [42]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=list(a_counts[a_counts<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [43]:
# Look at CLASSIFICATION value counts for binning
c_counts=application_df["CLASSIFICATION"].value_counts()
c_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [44]:
# You may find it helpful to look at CLASSIFICATION value counts >1
c_counts[c_counts>1000]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [45]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace=list(c_counts[c_counts<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [46]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df=pd.get_dummies(application_df)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [47]:
# Split our preprocessed data into our features and target arrays
y=application_df["IS_SUCCESSFUL"].values
X=application_df.drop(["IS_SUCCESSFUL"],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test,=train_test_split(X,y,random_state=48)

In [48]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [49]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input=len(X_train[0])
hl_1=10
hl_2=15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hl_1,input_dim=num_input,activation="relu")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hl_2,activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 10)                410       
                                                                 
 dense_13 (Dense)            (None, 15)                165       
                                                                 
 dense_14 (Dense)            (None, 1)                 16        
                                                                 
Total params: 591 (2.31 KB)
Trainable params: 591 (2.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [50]:
# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [51]:
# Train the model
fit_model=nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5920 - accuracy: 0.7041
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5619 - accuracy: 0.7248
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5576 - accuracy: 0.7272
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5560 - accuracy: 0.7294
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7302
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5542 - accuracy: 0.7305
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accuracy: 0.7304
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5531 - accuracy: 0.7304
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5521 - accuracy: 0.7308
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5524 - accura

In [52]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5558 - accuracy: 0.7258 - 463ms/epoch - 2ms/step
Loss: 0.5558344125747681, Accuracy: 0.7258309125900269


In [16]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Optimize the Model

In [53]:
# Look at ASK_AMT value counts for binning
ask_counts=application_df["ASK_AMT"].value_counts()
ask_counts

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [54]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
ask_amt_to_replace=list(ask_counts[ask_counts<20000].index)
ask_amt2_to_replace=list(ask_counts[ask_counts>20000].index)

# Replace in dataframe
for app in ask_amt_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(app,"<20000")
for app in ask_amt2_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(app,">20000")

# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

>20000    25398
<20000     8901
Name: ASK_AMT, dtype: int64

In [55]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input=len(X_train[0])
hl_1=10
hl_2=10
h1_3=10
h1_4=10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hl_1,input_dim=num_input,activation="relu")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hl_2,activation="relu")
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=h1_3,activation="relu")
)

# Fourth hidden layer
nn.add(
    tf.keras.layers.Dense(units=h1_4,activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 10)                410       
                                                                 
 dense_16 (Dense)            (None, 10)                110       
                                                                 
 dense_17 (Dense)            (None, 10)                110       
                                                                 
 dense_18 (Dense)            (None, 10)                110       
                                                                 
 dense_19 (Dense)            (None, 1)                 11        
                                                                 
Total params: 751 (2.93 KB)
Trainable params: 751 (2.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
# Train the model
fit_model=nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5469 - accuracy: 0.7352
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7355
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7352
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7364
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7354
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7363
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5461 - accuracy: 0.7364
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5468 - accuracy: 0.7352
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7350
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accura

In [59]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5606 - accuracy: 0.7255 - 381ms/epoch - 1ms/step
Loss: 0.5605642199516296, Accuracy: 0.7254810333251953
